In [ ]:
""" In this Jupyter notebook, we will explore the use of Spacy's PhraseMatcher with RapidFuzzy search to identify terms from a list of terms in a given text. Spacy is a popular open-source software library used for advanced Natural Language Processing (NLP) tasks, while RapidFuzzy is a fast and efficient fuzzy search library in Python.

The ability to quickly and accurately identify terms from a list in a text is a crucial task in NLP applications such as document classification, sentiment analysis, and named entity recognition. The combination of Spacy's PhraseMatcher and RapidFuzzy search can greatly simplify this task and reduce the computational time needed.

Throughout this notebook, we will walk through the steps to create a PhraseMatcher object, use RapidFuzzy search to compare the extracted phrases with a list of terms, and finally visualize the results. By the end of this notebook, you will have a better understanding of how to use Spacy and RapidFuzzy together to efficiently identify terms in a given text."""

In [ ]:
##Create term list should look like this [{"ID": 1, "term": "term1", "label": "label1"}, {"ID": 2, "term": "term2", "label": "label2"}, ... {"ID": 30000, "term": "term30000", "label": "label30000"}

import os
import pandas as pd
from multiprocessing  import Pool
import spacy
from spacy.matcher import PhraseMatcher
from rapidfuzz import fuzz


list_of_dictionaries = [dic for dic in os.listdir("DEBBIE_dictionaries_annotations-main/dictionaries") if dic.startswith(".") is False]

In [ ]:
terms = []
for dictionary in list_of_dictionaries:
    try:
        data = pd.read_csv("DEBBIE_dictionaries_annotations-main/dictionaries/"+dictionary,sep="\t",usecols=[0,1,2], names=['term', 'label',"id"], header=None)
        data = data.dropna()
        data = data.to_dict(orient="records")
        for d in data:
            d["label"] = d["label"].replace("LABEL=", "") # remove the "LABEL=" string from the "label" key
            d["id"] = d["id"].replace("ID=", "") # remove the "ID=" string from the "id" key
        terms += data 
    except:
        print(dictionary)

In [ ]:
def create_patterns(term):
    try:
        pattern = nlp(term["term"])
        fuzzy_pattern = [{"LOWER": token.lower_, "FUZZY": term["term"]} for token in pattern]
        return (fuzzy_pattern + [{"ID": term["id"], "LABEL": term["label"]}, pattern])
    except:
        print("error with term: "+str(term))

In [ ]:


nlp = spacy.load("en_core_web_sm") # load the small English model

matcher = PhraseMatcher(nlp.vocab) # initialize the PhraseMatcher object
#terms = [{"ID": 1, "term": "term1", "label": "label1"}, {"ID": 2, "term": "term2", "label": "label2"}, ... {"ID": 30000, "term": "term30000", "label": "label30000"}] # list of dictionaries containing terms to match (30000 terms)

# create a list of patterns that includes the original term and its fuzzy match, as well as the term ID and label
patterns = []
# for term in terms:
#     pattern = nlp(term["term"])
#     fuzzy_pattern = [{"LOWER": token.lower_, "FUZZY": term["term"]} for token in pattern]
#     patterns.append(fuzzy_pattern + [{"ID": term["id"], "LABEL": term["label"]}, pattern])
with Pool(6) as pool:
    patterns += pool.map(create_patterns,terms)
matcher.add("TERMS", None, *patterns)

# example text to match against
text = "Here is an example text that contains some of the terms we want to match."

doc = nlp(text) # create a Doc object from the example text
matches = []
for pattern in patterns:
    for match in matcher(doc):
        if match[0] == nlp.vocab.strings["TERMS"]:
            match_text = doc[match[1]:match[2]].text.lower()
            if fuzz.partial_ratio(match_text, pattern[0]["FUZZY"]) > 80:
                matches.append(match + pattern[-2:])

# print the matches
for match_id, start, end, metadata, _ in matches:
    print(f"Matched term: {doc[start:end].text}")
    print(f"ID: {metadata['ID']}")
    print(f"Label: {metadata['LABEL']}")
    print()


In [ ]:
terms[0]

In [8]:
patterns_2 = []

for n in terms:
    patter = {}
    pattern = []
    text = n["term"].split(" ")
    for n_t in text:
        pattern.append({"LOWER":n_t.lower()})
    patter["id"] = n["id"]
    patter["label"] = n["label"]
    patter["pattern"] = pattern
    patterns_2.append(patter)



In [5]:
ruler = nlp.add_pipe("entity_ruler",validate=True)


In [11]:
txt="""1998 Jan
Bioactive glass fiber/polymeric composites bond to bone tissue. 
Bioactive glass fibers were investigated for use as a fixation vehicle between a low modulus, polymeric composite and bone tissue. In an initial pilot study, bioactive glass fiber/polysulfone composites and all polysulfone control rods were implanted into the rabbit tibia; the study was subsequently expanded with implantation into the rabbit femur. Bone tissue exhibited direct contact with the glass fibers and adjacent polymer matrix and displayed a mechanical bond between the composite and bone tissue after six weeks implantation. Interfacial bond strengths after six weeks implantation averaged 12.4 MPa, significantly higher than those of the all polymer controls. Failure sites for the composite at six weeks generally occurred in the bone tissue or composite, whereas the failure site for the polymer implants occurred exclusively at the implant/tissue interface. The bioactive glass fiber/polysulfone composite achieved fixation to bone tissue through a triple mechanism: a bond to the bioactive glass fiber, mechanical interlocking between the tissue and glass fibers, and close apposition and possible chemical bond between the portions of the polymer and bone tissue. This last mechanism resulted from an overspill of bioactivity reactions from the fibers onto the surface of the surrounding polymer which we call the "halo" effect. """

In [9]:
ruler.add_patterns(patterns_2)

In [14]:
doc = nlp(txt)

In [12]:
nlp.remove_pipe("ner")

('ner', <spacy.pipeline.ner.EntityRecognizer at 0x7f230a90c350>)

In [15]:
for ent in doc.ents:
    print(ent.label_,ent.text)

EffectOnBiologicalSystem Bioactive
BiomaterialType glass
Structure fiber
BiomaterialType polymeric
Tissue bone tissue
EffectOnBiologicalSystem Bioactive
BiomaterialType glass
Structure fibers
BiomaterialType polymeric
BiomaterialType composite
Tissue bone tissue
EffectOnBiologicalSystem bioactive
BiomaterialType glass
Structure fiber
Biomaterial polysulfone
Biomaterial polysulfone
Species rabbit
MedicalApplication implantation
Species rabbit
Tissue femur
Tissue Bone tissue
BiomaterialType glass
Structure fibers
BiomaterialType polymer
BiomaterialType composite
Tissue bone tissue
MedicalApplication implantation
MedicalApplication implantation
BiomaterialType polymer
BiomaterialType composite
Tissue bone tissue
ResearchTechnique or
BiomaterialType composite
BiomaterialType polymer
ManufacturedObject implant
Tissue tissue
EffectOnBiologicalSystem bioactive
BiomaterialType glass
Structure fiber
Biomaterial polysulfone
BiomaterialType composite
Tissue bone tissue
EffectOnBiologicalSystem bi

In [ ]:
for d in data:
    if d["term"]:
        print("Empty value found for 'term' key in dictionary:", d)